In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn import svm
from bayes_opt import BayesianOptimization

In [10]:
#!pip install xgboost
#!pip install bayesian-optimization

In [11]:
features = pd.read_csv('features.csv')

In [12]:
features = features.drop('course_id',axis=1)
features[:5]

,thread_id,no_of_posts,no_of_comments,no_uni_users,no_of_anonymous_msg,staff_replied,no of msgs,avg_num_words,max_words,avg_resp_time,first_post_day,msg_rate,u_chain,index_longest_post,num_views,votes,votes_Square,index_max_votes,forum_id
0,2,9.0,2.0,5.0,2.0,0.0,11.0,12.454545,31.0,-152871.727273,2,1.359260e+09,True,4,277,2,4,0,10
1,2,9.0,2.0,5.0,2.0,0.0,11.0,12.454545,31.0,-152871.727273,2,1.359260e+09,True,4,277,0,0,0,10
2,2,9.0,2.0,5.0,2.0,0.0,11.0,12.454545,31.0,-152871.727273,2,1.359260e+09,True,4,277,1,1,0,10
3,2,9.0,2.0,5.0,2.0,0.0,11.0,12.454545,31.0,-152871.727273,2,1.359260e+09,True,4,277,0,0,0,10
4,2,9.0,2.0,5.0,2.0,0.0,11.0,12.454545,31.0,-152871.727273,2,1.359260e+09,True,4,277,0,0,0,10


In [13]:
features[["u_chain"]] *= 1

In [14]:
features[:5]

,thread_id,no_of_posts,no_of_comments,no_uni_users,no_of_anonymous_msg,staff_replied,no of msgs,avg_num_words,max_words,avg_resp_time,first_post_day,msg_rate,u_chain,index_longest_post,num_views,votes,votes_Square,index_max_votes,forum_id
0,2,9.0,2.0,5.0,2.0,0.0,11.0,12.454545,31.0,-152871.727273,2,1.359260e+09,1,4,277,2,4,0,10
1,2,9.0,2.0,5.0,2.0,0.0,11.0,12.454545,31.0,-152871.727273,2,1.359260e+09,1,4,277,0,0,0,10
2,2,9.0,2.0,5.0,2.0,0.0,11.0,12.454545,31.0,-152871.727273,2,1.359260e+09,1,4,277,1,1,0,10
3,2,9.0,2.0,5.0,2.0,0.0,11.0,12.454545,31.0,-152871.727273,2,1.359260e+09,1,4,277,0,0,0,10
4,2,9.0,2.0,5.0,2.0,0.0,11.0,12.454545,31.0,-152871.727273,2,1.359260e+09,1,4,277,0,0,0,10


## Preparing the Data

### forum_id 
    2: General (Miscellaneous) Discussion
    3: Assignments
    4: Study Groups / Meetups
    7: Course Feedback / Suggestions
    8: Lectures
    9: Platform Issues
    100: Signature Track
    otherwise: not remapped

In [15]:
X = features.drop('forum_id', axis=1)
y = features['forum_id']

In [16]:
# 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=109) 

In [17]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(591259, 18)
(147815, 18)
(591259,)
(147815,)


## SVM Model

In [18]:
'''from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train_scaled = scaling.transform(X_train)
X_test_scaled = scaling.transform(X_test)'''

'from sklearn.preprocessing import MinMaxScaler\nscaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)\nX_train_scaled = scaling.transform(X_train)\nX_test_scaled = scaling.transform(X_test)'

In [19]:
'''svm_model = svm.SVC(kernel='linear', C = 1.0)
svm_model.fit(X_train_scaled, y_train)'''

"svm_model = svm.SVC(kernel='linear', C = 1.0)\nsvm_model.fit(X_train_scaled, y_train)"

In [20]:
'''y_pred_svm = model.predict(X_test_scaled)'''

'y_pred_svm = model.predict(X_test_scaled)'

In [21]:
'''from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_svm))'''

'from sklearn.metrics import classification_report\nprint(classification_report(y_test,y_pred_svm))'

## XG Boost Model

In [23]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [24]:
y_pred = model.predict(X_test)

###  Creating the Classificationreport

In [48]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           2       0.81      0.79      0.80     34537
           3       0.79      0.91      0.85     38655
           4       0.96      0.92      0.94     12593
           7       0.91      0.44      0.60      3557
           8       0.78      0.73      0.75     22237
           9       0.87      0.48      0.62      3670
          10       0.90      0.91      0.91     32566

    accuracy                           0.83    147815
   macro avg       0.86      0.74      0.78    147815
weighted avg       0.84      0.83      0.83    147815



###  Creating the confusion Matrix and checking the accuracy

In [51]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[27261  4139   237    27  1776    89  1008]
 [ 1544 35186    41    32  1112    35   705]
 [  386   111 11644     2   160     3   287]
 [  579   581    18  1574   445    42   318]
 [ 2376  2836    50    48 16166    73   688]
 [  604   572    36    19   317  1758   364]
 [  996  1026   153    22   627    14 29728]]


In [50]:
accuracy = cm.diagonal().sum()/cm.sum()
print(accuracy)

0.8342658052295099


## Hyper Parameter Tuning on XGBoost

Parameters:

* learning_rate : Makes the model more robust by shrinking the weights on each step. Used to prevent overfitting.
* max_depth  : Determines how deeply each tree is allowed to grow during any boosting round.Range should be between 3-10. For starting, start from lower values.
* min_child_weight : Smaller values are chosen because leaf nodes could have small size groups.
* subsample, colsample_bytree  : ideal range is between 0.5 to 0.9. Low value can lead to underfitting
* n_estimators: number of trees you want to build.
* objective: determines the loss function to be used like reg:linear for regression problems, reg:logistic for classification problems with only decision, binary:logistic for classification problems with probability.

Regularization Parameters:
* gamma: controls whether a given node will split based on the expected reduction in loss after the split. A higher value leads to fewer splits.
* alpha: L1 regularization on leaf weights. A large value leads to more regularization.
* lambda: L2 regularization on leaf weights and is smoother than L1 regularization.

### Bayesian Optimization function for xgboost

In [66]:
def tuning_function(n_estimators ,learning_rate):
     params = {'n_estimators': int(n_estimators),
              'learning_rate':learning_rate,
              'subsample': 0.8,
              'eta': 0.1,
              'eval_metric': 'rmse',
             'max_depth':4,
             'min_child_weight':6,
             'gamma':0,
             'subsample':0.8,
             'colsample_bytree':0.8,
             'reg_alpha':0.005,
             'objective': 'logistic',
             'nthread':4,
             'seed':27,
             'eta':0.1}

In [74]:
bayes = BayesianOptimization(tuning_function, {'learning_rate':(0,1),
                                             'n_estimators':(100,120)
                                            })


In [83]:
#bayes.maximize(n_iter=5, init_points=8, acq='ei')

### Extracting the best parameters

In [77]:
params = bayes.max['params']
params['n_estimators']= int(params['n_estimators'])

In [78]:
hyper_model = xgb.XGBClassifier(**params).fit(X_train, y_train)

In [79]:
y_pred_HT = hyper_model.predict(X_test)

###  Creating the Classification Report

In [80]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_HT))

              precision    recall  f1-score   support

           2       0.82      0.81      0.82     34537
           3       0.81      0.92      0.86     38655
           4       0.96      0.93      0.95     12593
           7       0.92      0.50      0.65      3557
           8       0.80      0.75      0.77     22237
           9       0.90      0.53      0.67      3670
          10       0.91      0.92      0.92     32566

    accuracy                           0.85    147815
   macro avg       0.88      0.77      0.80    147815
weighted avg       0.85      0.85      0.85    147815



###  Creating the confusion Matrix and checking the accuracy

In [81]:
cm_hp = confusion_matrix(y_test, y_pred_HT)
print(cm_hp)

[[28042  3706   192    13  1639    78   867]
 [ 1453 35448    37    31  1047    29   610]
 [  344    90 11735     3   148     4   269]
 [  543   524    18  1784   376    32   280]
 [ 2279  2578    45    55 16620    64   596]
 [  548   527    45    23   286  1942   299]
 [  865   890   122    28   542     9 30110]]


In [82]:
accuracy = cm_hp.diagonal().sum()/cm.sum()
print(accuracy)

0.8502587694077056


#### From the XG Boost(Acc = 0.8343) and the hyper tunes XGBoost(Acc = 0.8503) accuracies, we can see that the accuracy of the model has increased after hypertuning by 0.2.